# Model Fitting & Optimization from Scratch (NumPy)

**Goal:** Implement a multi-layer neural network *from scratch* using NumPy to learn classifiers on MNIST and CIFAR. This Colab notebook guides students through writing activations, forward/backward passes, loss functions (cross-entropy, Huber/L1/L2), and implementing SGD with momentum and weight decay.

---

**Notes for instructors:**
- This notebook is structured with explanation cells and code cells. Students should fill the `TODO` sections.
- Designed to run in Google Colab. For CIFAR training you'll likely need GPU runtime to speed up.

---


## 1) Setup & Imports

Run the cell below to import libraries. If running in a fresh Colab environment, TensorFlow is available by default to load datasets. This notebook uses NumPy only for model math (no autograd).

In [ ]:
# Standard imports - this code is ready for Colab
import numpy as np
import matplotlib.pyplot as plt
from time import time
from typing import List, Tuple, Dict, Any

# For dataset loading (Colab has tensorflow installed)
try:
    from tensorflow.keras.datasets import mnist, cifar10
    tf_available = True
except Exception as e:
    print("TensorFlow dataset loader not available. You can still load datasets manually.")
    tf_available = False

np.random.seed(42)


## 2) Utilities

Helper functions for one-hot encoding, stable softmax, accuracy and plotting training curves.


1.  One-hot encoding	Convert integer labels to vector form for loss computation


2.   Stable Softmax	Safely convert logits to probabilities without overflow



1.   Accuracy	Measure model correctness
2.   Plot training curves	Visualize learning progress & detect over/underfitting




In [ ]:
def one_hot(labels: np.ndarray, num_classes: int) -> np.ndarray:
    y = np.zeros((labels.shape[0], num_classes), dtype=np.float32)
    for i, lab in enumerate(labels):
        y[i, int(lab)] = 1.0
    return y

def stable_softmax(x: np.ndarray) -> np.ndarray:
    # x: (batch, classes)
    x_max = np.max(x, axis=1, keepdims=True)
    ex = np.exp(x - x_max)
    return ex / np.sum(ex, axis=1, keepdims=True)

def accuracy(pred_probs: np.ndarray, labels: np.ndarray) -> float:
    preds = np.argmax(pred_probs, axis=1)
    return np.mean(preds == labels) * 100.0

def plot_training(history: Dict[str, List[float]]):
    epochs = len(history['train_loss'])
    fig, ax = plt.subplots(1,2, figsize=(12,4))
    ax[0].plot(range(1, epochs+1), history['train_loss'], label='train loss')
    ax[0].plot(range(1, epochs+1), history.get('val_loss', []), label='val loss')
    ax[0].set_xlabel('Epoch'); ax[0].set_ylabel('Loss'); ax[0].legend()
    ax[1].plot(range(1, epochs+1), history['train_acc'], label='train acc')
    ax[1].plot(range(1, epochs+1), history.get('val_acc', []), label='val acc')
    ax[1].set_xlabel('Epoch'); ax[1].set_ylabel('Accuracy (%)'); ax[1].legend()
    plt.show()

print('Utilities loaded.')

Utilities loaded.


## 3) Activations & Derivatives

Implement activations and their derivatives. Softmax is handled in the final layer; for backprop we provide a Jacobian helper for educational clarity (students may use alternative vectorized approaches).
Educational Notes for Students

Sigmoid

Squashes values to (0, 1) range.

Good for probabilities but suffers from vanishing gradients for large |x|.

Derivative:
𝜎
′
(
𝑥
)
=
𝜎
(
𝑥
)
(
1
−
𝜎
(
𝑥
)
)
σ
′
(x)=σ(x)(1−σ(x))

Tanh

Output in (-1, 1).

Centered, but still has vanishing gradient issues.

Derivative:
1
−
tanh
⁡
2
(
𝑥
)
1−tanh
2
(x)

ReLU

Zero for negative inputs, identity for positive.

Fast convergence but can cause "dead neurons".

Derivative: 0 if
𝑥
≤
0
x≤0, else 1.

Leaky ReLU

Fixes dead neuron issue by giving small slope (α) for negatives.

Softmax

Converts logits to probabilities for multi-class classification.

In backprop, for cross-entropy + softmax, derivative simplifies to:

∂
𝐿
∂
𝑧
=
softmax
(
𝑧
)
−
𝑦
one-hot
∂z
∂L
	​

=softmax(z)−y
one-hot
	​


so in practice you don’t need the full Jacobian.

In [ ]:
def linear(x): return x
def d_linear(out): return np.ones_like(out)

def sigmoid(x): return 1.0 / (1.0 + np.exp(-x))
def d_sigmoid(out): return out * (1.0 - out)

def tanh_act(x): return np.tanh(x)
def d_tanh(out): return 1.0 - out**2

def relu(x): return np.maximum(0, x)
def d_relu(out): return (out > 0).astype(np.float32)

def leaky_relu(x, alpha=0.01):
    return np.where(x >= 0, x, alpha * x)
def d_leaky_relu_out(out, alpha=0.01):
    # out is activated output; derivative needs original x, but we can reconstruct sign from out for leaky relu
    # For positive outputs derivative is 1, for negative it's alpha (approximately)
    deriv = np.ones_like(out)
    deriv[out < 0] = alpha
    return deriv

# Softmax forward is stable_softmax above.
# For backward, we'll provide a Jacobian computation for a single data point (educational)
def softmax_jacobian(s: np.ndarray) -> np.ndarray:
    # s is 1D softmax output vector (C,)
    s = s.reshape(-1,1)
    return np.diagflat(s) - s.dot(s.T)

def d_softmax(prev_grad: np.ndarray, softmax_out: np.ndarray) -> np.ndarray:
    # prev_grad: (batch, classes) gradient coming from next layer (dL/dy)
    # softmax_out: (batch, classes)
    # Compute dL/dz where z is pre-softmax inputs.
    batch = prev_grad.shape[0]
    out = np.zeros_like(prev_grad)
    for i in range(batch):
        J = softmax_jacobian(softmax_out[i])
        out[i] = prev_grad[i].dot(J)
    return out

# Activation dispatcher
ACT_FNS = {
    'linear': (linear, d_linear),
    'sigmoid': (sigmoid, d_sigmoid),
    'tanh': (tanh_act, d_tanh),
    'relu': (relu, d_relu),
    'leaky_relu': (leaky_relu, d_leaky_relu_out),
}

print('Activations ready.')

Activations ready.


## 4) Loss Functions

Implement cross-entropy (with softmax), L1, L2, and Huber losses and their derivatives.

In [ ]:
def cross_entropy_loss(probs: np.ndarray, targets: np.ndarray) -> float:
    # probs: (batch, classes) from softmax, targets: (batch, classes) one-hot
    # small epsilon for numerical stability
    eps = 1e-12
    probs_clipped = np.clip(probs, eps, 1. - eps)
    loss = -np.sum(targets * np.log(probs_clipped)) / probs.shape[0]
    return loss

def d_cross_entropy(probs: np.ndarray, targets: np.ndarray) -> np.ndarray:
    # derivative dL/dz where z is pre-softmax inputs if probs = softmax(z)
    # For cross-entropy with softmax, derivative simplifies to (probs - targets)/batch
    batch = probs.shape[0]
    return (probs - targets) / batch

def l2_loss(pred: np.ndarray, target: np.ndarray):
    diff = pred - target
    return 0.5 * np.mean(diff**2)

def d_l2(pred: np.ndarray, target: np.ndarray):
    batch = pred.shape[0]
    return (pred - target) / batch

def l1_loss(pred: np.ndarray, target: np.ndarray):
    return np.mean(np.abs(pred - target))

def d_l1(pred: np.ndarray, target: np.ndarray):
    diff = pred - target
    batch = pred.shape[0]
    # subgradient: sign(diff)
    return np.sign(diff) / batch

def huber_loss(pred: np.ndarray, target: np.ndarray, delta=1.0):
    diff = pred - target
    absd = np.abs(diff)
    mask = absd <= delta
    loss = np.where(mask, 0.5 * diff**2, delta * (absd - 0.5 * delta))
    return np.mean(loss)

def d_huber(pred: np.ndarray, target: np.ndarray, delta=1.0):
    diff = pred - target
    absd = np.abs(diff)
    mask = absd <= delta
    grad = np.where(mask, diff, delta * np.sign(diff))
    batch = pred.shape[0]
    return grad / batch

print('Loss functions ready.')

Loss functions ready.


## 5) Layer & Model Classes

Implement a simple Dense layer and Model class to handle forward/backward and parameter updates. Weight initialization options: Xavier (Glorot) and He.

In [ ]:
class DenseLayer:
    def __init__(self, input_dim:int, output_dim:int, activation='relu', init='xavier'):
        self.input_dim = input_dim
        self.output_dim = output_dim
        self.activation_name = activation
        self.activation, self.d_activation = ACT_FNS.get(activation, ACT_FNS['relu'])

        # weight init
        if init == 'xavier':
            limit = np.sqrt(6.0 / (input_dim + output_dim))
            self.W = np.random.uniform(-limit, limit, size=(input_dim, output_dim)).astype(np.float32)
        elif init == 'he':
            std = np.sqrt(2.0 / input_dim)
            self.W = np.random.randn(input_dim, output_dim).astype(np.float32) * std
        else:
            self.W = np.random.randn(input_dim, output_dim).astype(np.float32) * 0.01

        self.b = np.zeros((1, output_dim), dtype=np.float32)
        # gradients and velocity for momentum
        self.dW = np.zeros_like(self.W)
        self.db = np.zeros_like(self.b)
        self.vW = np.zeros_like(self.W)
        self.vb = np.zeros_like(self.b)

        # caches for backprop
        self.x = None
        self.z = None
        self.a = None

    def forward(self, x: np.ndarray) -> np.ndarray:
        # x: (batch, input_dim)
        self.x = x
        self.z = x.dot(self.W) + self.b  # (batch, output_dim)
        # apply activation (special-case softmax handled by Model)
        if self.activation_name == 'softmax':
            self.a = stable_softmax(self.z)
        else:
            # activation functions expect pre-activation input; some were implemented to accept x
            fn = ACT_FNS[self.activation_name][0]
            self.a = fn(self.z)
        return self.a

    def backward(self, grad_a: np.ndarray) -> np.ndarray:
        # grad_a: dL/da (batch, output_dim)
        # compute dL/dz first
        if self.activation_name == 'softmax':
            # use simplified derivative if combined with cross-entropy outside (handled elsewhere)
            # Otherwise use Jacobian-based derivative
            grad_z = d_softmax(grad_a, self.a)
        else:
            # derivative function expects activated output or z depending on implementation; we use a
            dfn = ACT_FNS[self.activation_name][1]
            grad_z = grad_a * dfn(self.a)

        # gradients w.r.t weights and bias
        batch = self.x.shape[0]
        self.dW = self.x.T.dot(grad_z)  # (input_dim, output_dim)
        self.db = np.sum(grad_z, axis=0, keepdims=True)
        # gradient w.r.t input to propagate to previous layer
        grad_x = grad_z.dot(self.W.T)
        return grad_x

class SimpleModel:
    def __init__(self, layers: List[DenseLayer]):
        self.layers = layers

    def forward(self, x: np.ndarray) -> np.ndarray:
        out = x
        for i, layer in enumerate(self.layers):
            out = layer.forward(out)
        return out

    def backward(self, grad_output: np.ndarray) -> None:
        grad = grad_output
        for layer in reversed(self.layers):
            grad = layer.backward(grad)

    def update(self, lr: float, momentum: float=0.9, decay: float=0.0):
        for layer in self.layers:
            # velocity update: v = m*v - (dW + decay*W)
            layer.vW = momentum * layer.vW - (layer.dW + decay * layer.W)
            layer.vb = momentum * layer.vb - (layer.db + decay * layer.b)
            # weights update: w += lr * v
            layer.W += lr * layer.vW * lr  # note: multiply by lr twice is a bug; we will correct below in comment
            layer.b += lr * layer.vb * lr

    def correct_update(self, lr: float, momentum: float=0.9, decay: float=0.0):
        # Correct implementation for students to use
        for layer in self.layers:
            layer.vW = momentum * layer.vW - lr * (layer.dW + decay * layer.W)
            layer.vb = momentum * layer.vb - lr * (layer.db + decay * layer.b)
            layer.W += layer.vW
            layer.b += layer.vb

print('Layer and Model classes defined.')

Layer and Model classes defined.


## 6) Training Loop

Fill in or run the training loop. We provide a skeleton: forward, compute loss, backward, update. Use `correct_update` in practice. For MNIST quick runs use small network and fewer epochs.

In [ ]:
def train_model(model: SimpleModel,
                X_train: np.ndarray, y_train: np.ndarray,
                X_val: np.ndarray, y_val: np.ndarray,
                epochs: int = 10, batch_size: int = 64,
                lr: float = 0.01, momentum: float = 0.9, decay: float = 0.0,
                loss_name: str = 'cross_entropy', verbose: bool = True):

    n = X_train.shape[0]
    history = {'train_loss': [], 'train_acc': [], 'val_loss': [], 'val_acc': []}

    for ep in range(1, epochs+1):
        perm = np.random.permutation(n)
        X_sh = X_train[perm]
        y_sh = y_train[perm]
        epoch_loss = 0.0
        epoch_acc = 0.0
        t0 = time()
        for i in range(0, n, batch_size):
            xb = X_sh[i:i+batch_size]
            yb = y_sh[i:i+batch_size]
            # Forward
            preds = model.forward(xb)  # if final layer is softmax this is probs
            # Loss and gradient
            if loss_name == 'cross_entropy':
                loss = cross_entropy_loss(preds, yb)
                grad = d_cross_entropy(preds, yb)  # dL/dz when softmax final
            elif loss_name == 'l2':
                loss = l2_loss(preds, yb)
                grad = d_l2(preds, yb)
            elif loss_name == 'l1':
                loss = l1_loss(preds, yb)
                grad = d_l1(preds, yb)
            elif loss_name == 'huber':
                loss = huber_loss(preds, yb)
                grad = d_huber(preds, yb)
            else:
                raise ValueError('Unknown loss')

            epoch_loss += loss * xb.shape[0]
            epoch_acc += accuracy(preds, np.argmax(yb, axis=1)) * xb.shape[0]

            # Backprop
            model.backward(grad)
            # Update weights
            model.correct_update(lr, momentum, decay)

        epoch_loss /= n
        epoch_acc /= n

        # Validation
        val_preds = model.forward(X_val)
        if loss_name == 'cross_entropy':
            val_loss = cross_entropy_loss(val_preds, y_val)
        else:
            val_loss = 0.0
        val_acc = accuracy(val_preds, np.argmax(y_val, axis=1))

        history['train_loss'].append(epoch_loss)
        history['train_acc'].append(epoch_acc)
        history['val_loss'].append(val_loss)
        history['val_acc'].append(val_acc)

        if verbose:
            print(f'Epoch {ep}/{epochs} - loss: {epoch_loss:.4f} - acc: {epoch_acc:.2f}% - val_loss: {val_loss:.4f} - val_acc: {val_acc:.2f}% - time: {time()-t0:.1f}s')

    return history

print('Training loop skeleton ready.')

Training loop skeleton ready.


## 7) Demo: MNIST Quick Setup

Load MNIST, preprocess, build a tiny model to test the pipeline. This cell is ready to run in Colab. For speed during debugging, use a subset of the data (e.g., 5000 train samples).

In [ ]:
if tf_available:
    (X_train, y_train), (X_test, y_test) = mnist.load_data()
    # flatten and normalize
    X_train = X_train.reshape(-1, 28*28).astype(np.float32) / 255.0
    X_test  = X_test.reshape(-1, 28*28).astype(np.float32) / 255.0
    # quick validation split
    X_val = X_train[-5000:]; y_val = y_train[-5000:]
    X_train = X_train[:-5000]; y_train = y_train[:-5000]
    # one hot encode
    y_train_oh = one_hot(y_train, 10)
    y_val_oh   = one_hot(y_val, 10)
    y_test_oh  = one_hot(y_test, 10)
    print('MNIST loaded:', X_train.shape, X_val.shape, X_test.shape)
else:
    print('TensorFlow datasets not available in this environment.')

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
MNIST loaded: (55000, 784) (5000, 784) (10000, 784)


In [ ]:
# Example model creation (to run in Colab)
# Create a small model: 784 -> 128(relu) -> 10(softmax)
# NOTE: For softmax final layer, we set activation_name to 'softmax' manually.
input_dim = 784
hidden = DenseLayer(input_dim, 128, activation='relu', init='he')
output = DenseLayer(128, 10, activation='softmax', init='xavier')

model = SimpleModel([hidden, output])
print('Model ready. Run training with train_model(...)')

Model ready. Run training with train_model(...)


## 8) Experiments & Hyperparameter Tuning

Try these experiments:
- Vary learning rate: [1e-3, 1e-2, 1e-1]
- Momentum: [0.0, 0.9, 0.99]
- Weight decay: [0.0, 1e-4, 1e-3]
- Activations: relu vs tanh vs sigmoid
- Loss functions: cross-entropy vs huber

Record training curves and final test accuracy. Try CIFAR after confirming MNIST works. For CIFAR expect to tune lr and use smaller network or more epochs.


In [ ]:
# =========================================================
# Imports & Utilities
# =========================================================
import numpy as np
import matplotlib.pyplot as plt
from time import time
from typing import List, Dict

try:
    from tensorflow.keras.datasets import mnist, cifar10
    tf_available = True
except:
    tf_available = False

np.random.seed(42)

# ---------- Utils ----------
def one_hot(labels: np.ndarray, num_classes: int) -> np.ndarray:
    y = np.zeros((labels.shape[0], num_classes), dtype=np.float32)
    for i, lab in enumerate(labels):
        y[i, int(lab)] = 1.0
    return y

def stable_softmax(x: np.ndarray) -> np.ndarray:
    x_max = np.max(x, axis=1, keepdims=True)
    ex = np.exp(x - x_max)
    return ex / np.sum(ex, axis=1, keepdims=True)

def accuracy(pred_probs: np.ndarray, labels: np.ndarray) -> float:
    preds = np.argmax(pred_probs, axis=1)
    return np.mean(preds == labels) * 100.0

# =========================================================
# Activations
# =========================================================
def linear(x): return x
def d_linear(out): return np.ones_like(out)

def sigmoid(x): return 1.0 / (1.0 + np.exp(-x))
def d_sigmoid(out): return out * (1.0 - out)

def tanh_act(x): return np.tanh(x)
def d_tanh(out): return 1.0 - out**2

def relu(x): return np.maximum(0, x)
def d_relu(out): return (out > 0).astype(np.float32)

def leaky_relu(x, alpha=0.01): return np.where(x >= 0, x, alpha * x)
def d_leaky_relu_out(out, alpha=0.01):
    deriv = np.ones_like(out)
    deriv[out < 0] = alpha
    return deriv

def softmax_jacobian(s: np.ndarray) -> np.ndarray:
    s = s.reshape(-1,1)
    return np.diagflat(s) - s.dot(s.T)

def d_softmax(prev_grad: np.ndarray, softmax_out: np.ndarray) -> np.ndarray:
    batch = prev_grad.shape[0]
    out = np.zeros_like(prev_grad)
    for i in range(batch):
        J = softmax_jacobian(softmax_out[i])
        out[i] = prev_grad[i].dot(J)
    return out

ACT_FNS = {
    'linear': (linear, d_linear),
    'sigmoid': (sigmoid, d_sigmoid),
    'tanh': (tanh_act, d_tanh),
    'relu': (relu, d_relu),
    'leaky_relu': (leaky_relu, d_leaky_relu_out),
}

# =========================================================
# Losses
# =========================================================
def cross_entropy_loss(probs: np.ndarray, targets: np.ndarray) -> float:
    eps = 1e-12
    probs_clipped = np.clip(probs, eps, 1. - eps)
    return -np.sum(targets * np.log(probs_clipped)) / probs.shape[0]

def d_cross_entropy(probs: np.ndarray, targets: np.ndarray) -> np.ndarray:
    batch = probs.shape[0]
    return (probs - targets) / batch

def huber_loss(pred: np.ndarray, target: np.ndarray, delta=1.0):
    diff = pred - target
    absd = np.abs(diff)
    mask = absd <= delta
    loss = np.where(mask, 0.5 * diff**2, delta * (absd - 0.5 * delta))
    return np.mean(loss)

def d_huber(pred: np.ndarray, target: np.ndarray, delta=1.0):
    diff = pred - target
    absd = np.abs(diff)
    mask = absd <= delta
    grad = np.where(mask, diff, delta * np.sign(diff))
    batch = pred.shape[0]
    return grad / batch

# =========================================================
# Layers & Model
# =========================================================
class DenseLayer:
    def __init__(self, input_dim:int, output_dim:int, activation='relu', init='xavier'):
        self.input_dim = input_dim
        self.output_dim = output_dim
        self.activation_name = activation
        self.activation, self.d_activation = ACT_FNS.get(activation, ACT_FNS['relu'])

        # init weights
        if init == 'xavier':
            limit = np.sqrt(6.0 / (input_dim + output_dim))
            self.W = np.random.uniform(-limit, limit, size=(input_dim, output_dim)).astype(np.float32)
        elif init == 'he':
            std = np.sqrt(2.0 / input_dim)
            self.W = np.random.randn(input_dim, output_dim).astype(np.float32) * std
        else:
            self.W = np.random.randn(input_dim, output_dim).astype(np.float32) * 0.01

        self.b = np.zeros((1, output_dim), dtype=np.float32)

        # gradients & velocity
        self.dW = np.zeros_like(self.W)
        self.db = np.zeros_like(self.b)
        self.vW = np.zeros_like(self.W)
        self.vb = np.zeros_like(self.b)

        self.x = None
        self.z = None
        self.a = None

    def forward(self, x: np.ndarray) -> np.ndarray:
        self.x = x
        self.z = x.dot(self.W) + self.b
        if self.activation_name == 'softmax':
            self.a = stable_softmax(self.z)
        else:
            fn = ACT_FNS[self.activation_name][0]
            self.a = fn(self.z)
        return self.a

    def backward(self, grad_a: np.ndarray) -> np.ndarray:
        if self.activation_name == 'softmax':
            grad_z = d_softmax(grad_a, self.a)
        else:
            dfn = ACT_FNS[self.activation_name][1]
            grad_z = grad_a * dfn(self.a)

        self.dW = self.x.T.dot(grad_z)
        self.db = np.sum(grad_z, axis=0, keepdims=True)
        grad_x = grad_z.dot(self.W.T)
        return grad_x

class SimpleModel:
    def __init__(self, layers: List[DenseLayer]):
        self.layers = layers

    def forward(self, x: np.ndarray) -> np.ndarray:
        out = x
        for layer in self.layers:
            out = layer.forward(out)
        return out

    def backward(self, grad_output: np.ndarray) -> None:
        grad = grad_output
        for layer in reversed(self.layers):
            grad = layer.backward(grad)

    def correct_update(self, lr: float, momentum: float=0.9, decay: float=0.0):
        for layer in self.layers:
            layer.vW = momentum * layer.vW - lr * (layer.dW + decay * layer.W)
            layer.vb = momentum * layer.vb - lr * (layer.db + decay * layer.b)
            layer.W += layer.vW
            layer.b += layer.vb

# =========================================================
# Training Loop
# =========================================================
def train_model(model: SimpleModel,
                X_train: np.ndarray, y_train: np.ndarray,
                X_val: np.ndarray, y_val: np.ndarray,
                epochs: int = 10, batch_size: int = 64,
                lr: float = 0.01, momentum: float = 0.9, decay: float = 0.0,
                loss_name: str = 'cross_entropy', verbose: bool = True):

    n = X_train.shape[0]
    history = {'train_loss': [], 'train_acc': [], 'val_loss': [], 'val_acc': []}

    for ep in range(1, epochs+1):
        perm = np.random.permutation(n)
        X_sh = X_train[perm]
        y_sh = y_train[perm]
        epoch_loss, epoch_acc = 0.0, 0.0
        t0 = time()

        for i in range(0, n, batch_size):
            xb = X_sh[i:i+batch_size]
            yb = y_sh[i:i+batch_size]
            preds = model.forward(xb)

            if loss_name == 'cross_entropy':
                loss = cross_entropy_loss(preds, yb)
                grad = d_cross_entropy(preds, yb)
            elif loss_name == 'huber':
                loss = huber_loss(preds, yb)
                grad = d_huber(preds, yb)
            else:
                raise ValueError('Unknown loss')

            epoch_loss += loss * xb.shape[0]
            epoch_acc += accuracy(preds, np.argmax(yb, axis=1)) * xb.shape[0]

            model.backward(grad)
            model.correct_update(lr, momentum, decay)

        epoch_loss /= n
        epoch_acc /= n
        val_preds = model.forward(X_val)
        val_loss = cross_entropy_loss(val_preds, y_val) if loss_name=='cross_entropy' else huber_loss(val_preds,y_val)
        val_acc = accuracy(val_preds, np.argmax(y_val, axis=1))

        history['train_loss'].append(epoch_loss)
        history['train_acc'].append(epoch_acc)
        history['val_loss'].append(val_loss)
        history['val_acc'].append(val_acc)

        if verbose:
            print(f'Epoch {ep}/{epochs} - loss: {epoch_loss:.4f} - acc: {epoch_acc:.2f}% - val_loss: {val_loss:.4f} - val_acc: {val_acc:.2f}% - time: {time()-t0:.1f}s')

    return history

# =========================================================
# Load MNIST
# =========================================================
if tf_available:
    (X_train, y_train), (X_test, y_test) = mnist.load_data()
    X_train = X_train.reshape(-1, 28*28).astype(np.float32) / 255.0
    X_test  = X_test.reshape(-1, 28*28).astype(np.float32) / 255.0
    X_val = X_train[-5000:]; y_val = y_train[-5000:]
    X_train = X_train[:-5000]; y_train = y_train[:-5000]
    y_train_oh = one_hot(y_train, 10)
    y_val_oh   = one_hot(y_val, 10)
    y_test_oh  = one_hot(y_test, 10)
    print('MNIST loaded:', X_train.shape, X_val.shape, X_test.shape)
else:
    print("TensorFlow datasets not available.")

# =========================================================
# Hyperparameter Experiments
# =========================================================
import pandas as pd
results = []

lrs = [1e-3, 1e-2, 1e-1]
momentums = [0.0, 0.9, 0.99]
decays = [0.0, 1e-4, 1e-3]
acts = ['relu', 'tanh', 'sigmoid']
losses = ['cross_entropy', 'huber']

for lr in lrs:
    for mom in momentums:
        for wd in decays:
            for act in acts:
                for loss in losses:
                    print(f"\n>>> Training with lr={lr}, mom={mom}, wd={wd}, act={act}, loss={loss}")
                    model = SimpleModel([
                        DenseLayer(784, 128, activation=act, init='he'),
                        DenseLayer(128, 10, activation='softmax', init='xavier')
                    ])
                    history = train_model(model,
                                          X_train, y_train_oh,
                                          X_val, y_val_oh,
                                          epochs=5,  # adjust to 10+ for full runs
                                          lr=lr, momentum=mom, decay=wd,
                                          loss_name=loss, verbose=False)
                    val_acc = history['val_acc'][-1]
                    test_acc = accuracy(model.forward(X_test), y_test)
                    results.append({
                        'lr': lr, 'momentum': mom, 'decay': wd,
                        'activation': act, 'loss': loss,
                        'val_acc': val_acc, 'test_acc': test_acc
                    })
                    print(f"Val acc={val_acc:.2f}% | Test acc={test_acc:.2f}%")

df = pd.DataFrame(results)
print("\n===== Summary of Experiments =====")
print(df.sort_values(by='val_acc', ascending=False).head(10))


MNIST loaded: (55000, 784) (5000, 784) (10000, 784)

>>> Training with lr=0.001, mom=0.0, wd=0.0, act=relu, loss=cross_entropy
Val acc=38.66% | Test acc=37.36%

>>> Training with lr=0.001, mom=0.0, wd=0.0, act=relu, loss=huber
Val acc=49.40% | Test acc=49.48%

>>> Training with lr=0.001, mom=0.0, wd=0.0, act=tanh, loss=cross_entropy
Val acc=54.00% | Test acc=52.52%

>>> Training with lr=0.001, mom=0.0, wd=0.0, act=tanh, loss=huber
Val acc=50.48% | Test acc=50.16%

>>> Training with lr=0.001, mom=0.0, wd=0.0, act=sigmoid, loss=cross_entropy
Val acc=20.40% | Test acc=18.80%

>>> Training with lr=0.001, mom=0.0, wd=0.0, act=sigmoid, loss=huber
Val acc=19.08% | Test acc=19.14%

>>> Training with lr=0.001, mom=0.0, wd=0.0001, act=relu, loss=cross_entropy
Val acc=50.22% | Test acc=46.99%

>>> Training with lr=0.001, mom=0.0, wd=0.0001, act=relu, loss=huber
Val acc=45.96% | Test acc=44.21%

>>> Training with lr=0.001, mom=0.0, wd=0.0001, act=tanh, loss=cross_entropy
Val acc=61.18% | Test acc=

## 9) Submission

Turn in the Colab notebook (`.ipynb`) with the following:
- Completed implementations (no TODOs left)
- Training plots and short answers to observations (learning curves, stability issues)
- `activations.py`, `model.py` (if you split code into files) and `questions.txt` explaining your experiments and results.

---

**Instructor hint:** you may add unit tests that run small shapes through activations and gradients to verify correctness automatically.